In [2]:
# from flask_other.app import app
from flask_socketio import disconnect as dc
from flask import Flask
from flask_socketio import SocketIO, emit,join_room,leave_room
from flask_socketio import disconnect
from threading import Lock
import os,sys

app=Flask("demo")
from flask import  render_template,request
import time
import re

import pandas as pd
import numpy as np
import time
import re
import requests

import atexit

from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.interval import IntervalTrigger

logger = Logger('Websocket-Flask',level=ENV.NODE_LOG_LEVEL.value).logger



##################################
async_mode = None
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app, async_mode=async_mode)

thread = None
thread_lock = Lock()
clients = []
sentences = []

name_space = '/chat'

@app.route('/')
def index():
    return render_template('index.html', async_mode=socketio.async_mode)


@socketio.on('client_send',namespace=name_space)
def client_msg(msg):  
    uid = request.sid
    sentence = msg.get('data')
    sentence = decode(sentence)
    sentences.append(sentence)
    response = cache.chat(uid, sentence)
    if response == 'end':
        socketio.emit('my_response',{'data':'感谢选择江苏逸能，再见！'},room = uid, namespace=name_space)
        disconnect_frontend(uid)
    elif response is None:
        socketio.emit('my_response',{'data':'当前会话已经过期，感谢选择江苏逸能，再见！'},room = uid, namespace=name_space)
        disconnect_frontend(uid)
    else:
        socketio.emit('my_response',{'data':response},room = uid, namespace=name_space)

def decode(msg):
    msg = re.sub(r'%u', r'\u', msg)
    msg = msg.encode('latin-1').decode('unicode_escape')
    return msg

#sned message to a specific user
def unique_message(uid):
    socketio.emit('unique_messgae',{'data':'are you still here'}, room=uid, namespace=name_space)
    
def disconnect_frontend(uid):
    socketio.emit('my_response',{'status':'disconnected'},room = uid, namespace=name_space)


@socketio.on('disconnect',namespace=name_space)
def disconnect():
    uid = request.sid
    cache.remove_session(uid)
    leave_room(uid)
    dc()
    logger.info('{} has been disconnected from API'.format(uid))
    
    
@socketio.on('connect',namespace=name_space)
def connect():
    
    uid = request.sid
    clients.append(uid)
    logger.info('{} is trying to connect!'.format(uid))
    
    join_room(uid)
    if cache.create_session(uid):
        logger.info('{} join connection successfully'.format(uid))
        response = cache.chat(uid, '')
        if response is not None:
            socketio.emit('my_response',{'data':response},room = uid,namespace=name_space) #the first sentence
            return None
        
    else:
        logger.info('{} cannot join connection'.format(uid))
        socketio.emit('my_response',{'data':'server busy. please click new conv'},room = uid,namespace=name_space) 
        disconnect_frontend(uid)
    



In [3]:
print('http://10.0.24.31:8899/')
print('http://0.0.0.0:6006/')
scheduler = BackgroundScheduler()
scheduler.start()
scheduler.add_job(
    func=cache.purge_inactive,
    trigger=IntervalTrigger(seconds=3),
    id='purge_cache',
    name='purge_inactive',
    replace_existing=True)
# Shut down the scheduler when exiting the app
atexit.register(lambda: scheduler.shutdown())

socketio.run(app,'0.0.0.0',6006)

http://10.0.24.31:8899/
http://0.0.0.0:6006/


2018-07-18 02:29:18,173 - INFO - CLASS:Websocket-Flask- METHOD:connect -LINE:90 - MSG:a2cd3df06623414db9e85cc504a74216 is trying to connect!
2018-07-18 02:29:18,746 - INFO - CLASS:Cache- METHOD:create_session -LINE:42 - MSG:New session was created: a2cd3df06623414db9e85cc504a74216
2018-07-18 02:29:18,747 - INFO - CLASS:Cache- METHOD:create_session -LINE:43 - MSG:Remaining session number is: 999
2018-07-18 02:29:18,752 - INFO - CLASS:Websocket-Flask- METHOD:connect -LINE:94 - MSG:a2cd3df06623414db9e85cc504a74216 join connection successfully
2018-07-18 02:29:18,755 - INFO - CLASS:Cache- METHOD:chat -LINE:67 - MSG:receive message from user: a2cd3df06623414db9e85cc504a74216 ====================
2018-07-18 02:29:18,757 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:376 - MSG:Current node name is s0
2018-07-18 02:29:18,758 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:381 - MSG:Output label is 0
2018-07-18 02:29:18,764 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:391 - MSG:Next node

In [1]:
import time
import gc
import sys, os

ENV_PATH = '../../../ENV/'
LOG_PATH = '../../../Lib/'
sys.path.append(ENV_PATH)
sys.path.append(LOG_PATH)
from env import ENV
from LOG import Logger



class Cache:
    def __init__(self, graph_path,msg_path,model_dict,max_session=1000,debug=False):
        self.max_session = 1000
        self.inform_interval = 60
        self.inactive_maxlength = 150
        #{'uid': {'strategy': Tree(), 'time_response': <time>, 'time_inform': <>}
        self.active_session = {}
        self.model_dict = model_dict
        self.graph_path = graph_path
        self.msg_path = msg_path
        self.debug=debug
        self.log = Logger(self.__class__.__name__,level=ENV.NODE_LOG_LEVEL.value).logger
        self._print()
        
    def _print(self):
        self.log.info('Max num of session is: {}'.format(self.max_session))
        self.log.info('inform inacitve interval is {} seconds'.format(self.inform_interval))
        self.log.info('inactive max length is {} seconds'.format(self.inactive_maxlength))
        if self.debug:
            self.log.info('DEBUG is enabled')
        
        
    def create_session(self, uid, profile=None):
        if len(self.active_session) < self.max_session:
            self.active_session[uid] = {}
            try:
                self.active_session[uid].update({'strategy':TreeStage1(graph_path=self.graph_path,
                                                                       msg_path=self.msg_path,
                                                                       debug=self.debug,
                                                                       profile=profile)})
            except KeyError as e:
                self.log.error('Key {} does not exist in profile'.format(e))
                self.log.error('create session for user {} failed'.format(uid))
                return False
                
            self.active_session[uid].update({'time_response':time.time()})
            self.active_session[uid].update({'time_inform':time.time()})
            self.active_session[uid].update({'chatting':[]})
            self.log.info('New session was created: {}'.format(uid))
            self.log.info('Remaining session number is: {}'.format(self.max_session-len(self.active_session)))
            return True
        else:
            return False
        
    def remove_session(self,uid):
        response = '您当前的会话超过 {} 秒没有响应，系统将关闭当前会话！如有需求，请开始新的对话！'.format(self.inactive_maxlength)
        try:
            socketio.emit('my_response',{'data':response},room = uid, namespace=name_space)
        except:
            pass
        try:
            disconnect_frontend(uid)
        except:
            pass
        try:
            del self.active_session[uid]
            self.log.info('{} session is inactive, it has been removed!'.format(uid))
        except KeyError:
                pass
        gc.collect()
            
    def chat(self,uid,sentence):
        if self.active_session.get(uid) is not None:
            self.log.info('receive message from user: {} ===================='.format(uid))
            response = self.active_session[uid]['strategy'].process(sentence, self.model_dict)
            self.active_session[uid]['time_response'] = time.time()
            self.active_session[uid]['time_inform'] = time.time()
            self.active_session[uid]['chatting'].append(response)
            self.log.info('processing messages for user {} has been done!----------------'.format(uid))
        else:
            response = None
        return response
        
    
    
    def _bulk_deletes(self):
        current = time.time()
        remove_list = []
        try:
            for uid in self.active_session:
                try:
                    if current - self.active_session[uid]['time_response'] > self.inactive_maxlength:
                        remove_list.append(uid)
                except KeyError:
                    pass
        except RuntimeError as e:
            self.log.error(e)
            return False
        finally:
            # delete
            for uid in remove_list:              
                self.remove_session(uid)
        return True
    
    def _bulk_inform(self):
        current = time.time()
        inform_list = []
        try:
            for uid in self.active_session:
                try:
                    if current - self.active_session[uid]['time_inform'] > self.inform_interval:
                        inform_list.append(uid)
                     
                except KeyError as e:
                        self.log.error(e)
                        pass
            for each in inform_list:
                self.inform_inactive(each)
            
        except RuntimeError:
            
            return False
        
        return True
    
    
    def purge_inactive(self):
        current = time.time()
        while True:
            if self._bulk_deletes():
                break

        while True:
            if self._bulk_inform():
                break
        
                
        
    def inform_inactive(self, uid):
        self.active_session[uid]['time_inform'] = time.time()
        response = '您有在听我说吗?请回答我刚才的问题！'
        socketio.emit('my_response',{'data':response},room = uid, namespace=name_space)
        self.log.info('{} is inactive. Just inform that user'.format(uid))
          
################################################
import sys,os
sys.path.append('../../../MLModel/code/OneClickTraining/')
sys.path.append('../../../MLModel/code/TreeModelV2/')
from all_model_py import *
import pickle
from chatbotv1 import *



models_list = ['IDClassifier','CutDebt','IfKnowDebtor','WillingToPay','Installment','ConfirmLoan']
savedModel_path = '../../../MLModel/savedModel/{}/{}.pickle'
need_set_TIMEZONE = ['CutDebt','WillingToPay','Installment','ConfirmLoan']

model_dict = {}
for each_model in models_list:
    model_dict[each_model] = pickle.load(open(savedModel_path.format(each_model,each_model), 'rb'))
    model_dict[each_model].classify('再说一次')
    if each_model in need_set_TIMEZONE:
        model_dict[each_model].re_time._set_timeZone()
    
model_dict['StopClassifier'] = StopClassifier()
model_dict['InitClassifier'] = InitClassifier()    

graph_path='../../../MLModel/data/TreeModel/treeConnection.csv'
msg_path='../../../MLModel/data/TreeModel/node_message.csv'
#################################################################
cache = Cache(graph_path=graph_path,
              msg_path=msg_path,
              model_dict=model_dict,debug=True)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.821 seconds.
Prefix dict has been built succesfully.


Time Zone is set from ENV: Asia/Shanghai
Time Zone is set from ENV: Asia/Shanghai


2018-07-20 03:10:46,449 - INFO - CLASS:Cache- METHOD:_print -LINE:29 - MSG:Max num of session is: 1000
2018-07-20 03:10:46,450 - INFO - CLASS:Cache- METHOD:_print -LINE:30 - MSG:inform inacitve interval is 60 seconds
2018-07-20 03:10:46,452 - INFO - CLASS:Cache- METHOD:_print -LINE:31 - MSG:inactive max length is 150 seconds
2018-07-20 03:10:46,454 - INFO - CLASS:Cache- METHOD:_print -LINE:33 - MSG:DEBUG is enabled


Time Zone is set from ENV: Asia/Shanghai
Time Zone is set from ENV: Asia/Shanghai


In [2]:
cache.create_session(1000)

2018-07-20 03:10:47,113 - DEBUG - CLASS:PF- METHOD:_load_default -LINE:336 - MSG:profile is None. The default demo profile will be loaded!
2018-07-20 03:10:47,118 - INFO - CLASS:PF- METHOD:_load_default -LINE:357 - MSG:Customer ID is 1000000000, principal is 50,000, apr is 9%
2018-07-20 03:10:47,548 - INFO - CLASS:Cache- METHOD:create_session -LINE:52 - MSG:New session was created: 1000
2018-07-20 03:10:47,549 - INFO - CLASS:Cache- METHOD:create_session -LINE:53 - MSG:Remaining session number is: 999


True

In [3]:
t = cache.active_session[1000]['strategy']

In [10]:
n = t.nodes['cf_s1_n1_identity_q']

In [11]:
n.output_label

109

In [9]:
t.process('',model_dict)

2018-07-20 03:11:51,444 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:534 - MSG:Current node name is cf_s1_n1_identity_q
2018-07-20 03:11:51,460 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:539 - MSG:Output label is 109
2018-07-20 03:11:51,465 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:550 - MSG:Next node name is cf_s1_n1_identity_q.


'请你不要说这些有的没的，您是李明先生吗？<-current node is: cf_s1_n1_identity_q-><-output label is: 109-><-next node is: cf_s1_n1_identity_q->'

In [4]:
c = model_dict['IDClassifier']
c.classify('天气不错')

{'label': 109, 'pred_prob': array([[0.01752875, 0.01967167, 0.96279958],
        [0.04673296, 0.05362745, 0.8996396 ],
        [0.02181207, 0.02078342, 0.9574045 ]]), 'av_pred': array([0.02869126, 0.03136085, 0.93994789])}

In [6]:
cache.create_session(111,{'name':'wei'})

2018-07-19 22:38:00,262 - DEBUG - CLASS:PF- METHOD:_load_profile -LINE:360 - MSG:Loading From Profile
2018-07-19 22:38:00,262 - DEBUG - CLASS:PF- METHOD:_load_profile -LINE:360 - MSG:Loading From Profile
2018-07-19 22:38:00,262 - DEBUG - CLASS:PF- METHOD:_load_profile -LINE:360 - MSG:Loading From Profile
2018-07-19 22:38:00,270 - ERROR - CLASS:Cache- METHOD:create_session -LINE:45 - MSG:Key 'principal' does not exist in profile
2018-07-19 22:38:00,270 - ERROR - CLASS:Cache- METHOD:create_session -LINE:45 - MSG:Key 'principal' does not exist in profile
2018-07-19 22:38:00,270 - ERROR - CLASS:Cache- METHOD:create_session -LINE:45 - MSG:Key 'principal' does not exist in profile
2018-07-19 22:38:00,278 - ERROR - CLASS:Cache- METHOD:create_session -LINE:46 - MSG:create session for user 111 failed
2018-07-19 22:38:00,278 - ERROR - CLASS:Cache- METHOD:create_session -LINE:46 - MSG:create session for user 111 failed
2018-07-19 22:38:00,278 - ERROR - CLASS:Cache- METHOD:create_session -LINE:46 

False

In [8]:
model_dict['WillingToPay'].classify('我可以用支付宝还么？')

{'label': 0,
 'pred_prob': array([[0.61363318, 0.06782953, 0.28882034, 0.02971695],
        [0.56661775, 0.11023825, 0.28412199, 0.03902201],
        [0.37452761, 0.09958758, 0.49513153, 0.03075327]]),
 'av_pred': array([0.51825952, 0.09255179, 0.35602462, 0.03316408]),
 'time_extract': []}

In [10]:
t = TreeStage1(graph_path=graph_path,msg_path=msg_path,debug=True)

In [12]:

print(t.process('我可以用支付宝还么？',model_dict))


您好，您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5万2千块钱，其中利息1500，滞纳金500块。请问为什么到现在都没有处理呢？
current node is: cf_s1_n1_identity_q
output label is: 0
next node is: cf_s1_n2_confirmLoan_q


In [17]:
model = model_dict['WillingToPay']

In [2]:
cache.create_session(111)

True

In [4]:
cache.chat(111,'是的')

'您好，您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5万2千块钱，其中利息1500，滞纳金500块。请问为什么到现在都没有处理呢？\ncurrent node is: cf_s1_n1_identity_q\noutput label is: 0\nnext node is: cf_s1_n2_confirmLoan_q'

In [1]:
a= {'k':1}

In [2]:
a['b']

KeyError: 'b'